## Import Libraries

In [1]:
import os
import sys

## Read text from PDF
##### Provide the path of PDF file before initialized everything

In [2]:
data='data.pdf'

In [3]:
data

'data.pdf'

## Extract data by reading

In [4]:
import PyPDF2
from docx import Document

In [5]:
def extract_text_pdf(pdf):
    text = ""
    reader = PyPDF2.PdfReader(pdf)
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

In [6]:
def extract_text_docx(docx):
    doc = Document(docx)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

In [7]:
'''def get_pdf_text(pdf_docs):
    text =""
    for pdf in pdf_docs:
        pdf_reader =PyPDF2.PdfReader(pdf_docs) 
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text'''



'''The file can be any pdf or any docs. read the pdf/docs and go through each and every pages and extract the text'''


'The file can be any pdf or any docs. read the pdf/docs and go through each and every pages and extract the text'

In [8]:
def extract(doc):
    _, file_extension = doc.split('.')
    print(file_extension)
    if file_extension == 'pdf':
        return extract_text_pdf(doc)
    elif file_extension == 'docx':
        return extract_text_docx(doc)
    else:
        raise ValueError("Unsupported file format")

In [9]:
text_data=extract(data)


pdf


In [10]:
#print(text_data)

### Convert each text into chunks
##### need to split the text using CharacterTextSplitter (which is basically present in Langchain.text_splitter) such that it should not increase token size


In [11]:
from langchain.text_splitter import CharacterTextSplitter

In [12]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size =1300,
    chunk_overlap =300,
    length_function =len,

    )

In [13]:
chunks = text_splitter.split_text(text_data)

#taking the particular text & splitting based on information

In [14]:
len(chunks)

60

In [41]:
print(chunks[10])

Evaluation Method:  Practical Examination 30% and End Semester Examination 
70% (Student must obtain at least 35% from  the theory paper and 30% from  
the practical paper)  
 
Level  I - Semester  II 
COM121 β: Data  Structures  and  Algorithms  (30 lecture  hrs.  & 22.5  practical 
hrs.)  Introduction to Data Structures, Stacks, Queues and Lists, Trees, Graphs, Running 
Time of an Algorithm, Sorting Algorithms, Searching Algorithms.  Practical using Java/C 
Evaluation  Method:  Practical  Examination  30%  and End Semester  Examination 
70%  (Student  must  obtain  at least  35%  from  the theory  paper  and 30%  from 
the practical paper)  
 
COM122 β:  File organization  & DBMS  (30 lecture  hrs.  & 22.5  practical  hrs.) 
File Organization:  Introduction  to file organization,  Storage  devices,  Record  structure 
and design,  Data  transferring  times,  Traditional  file organizations  (File,  Sequential,  Indexed, 
Indexed -sequential  and Hashed  Files),  Accessing  the physic

In [43]:
print(chunks)

['University of Ruhuna \nFaculty of Science \nMatara, Sri Lanka  \n \nStudent  Handbook  - 2020  \n \n \n \nBachelor  of Science \nAnd  \nBachelor of Computer Science \n(General and Honours) Degree \nProgrammes  \n \n2019/2020\n2   \n \nDepartment  of Computer  Science  53 \n1.1 Research  Areas  ................................ ................................ ................................ ...............  53 \n1.2 Head  of the Department  ................................ ................................ ................................  54 \n1.3 Members  of Academic  Staff ................................ ................................ ...........................  54 \n1.4 Members  of the Academic  Support  Staff ................................ ................................ ..... 55 \n1.5 Course  Units  in Computer  Science  for B.Sc.  (General)  Degree  ..............................  55 \n1.6 ICT Course  units  of Bachelor  of Science  (General)  Degree  ...................

## Convert chunks into vectors(Text Embeddings)

In [17]:
import os
#Import 'OpenAIEmbeddings class'
from langchain.embeddings import OpenAIEmbeddings

from langchain_community.vectorstores import FAISS
#initialize the connection of my db

# Load environment variables from the .env file
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv()) 

# Retrieve the OpenAI API key from the environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# Ensure the API key is available
if OPENAI_API_KEY is None:
    raise ValueError("OpenAI API key not found. Please set it in the .env file.")

# Initialize OpenAIEmbeddings with your API key
embedding = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)



c:\Users\acer\OneDrive\Desktop\QA\AI_QA\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [18]:
#store in the vector db 
db = FAISS.from_texts(texts = chunks,embedding = embedding)

In [19]:
if os.path.exists("vectorstore/index.pkl"):
    # Load the existing vector store
    db = FAISS.load_local("vectorstore", embedding,allow_dangerous_deserialization=True)
else:
    # Create a new vector store from the text chunks
    db = FAISS.from_texts(texts=chunks, embedding=embedding)
    # Save the newly created vector store locally
    db.save_local("vectorstore")

### Similarity Check

In [20]:
#query = "what are the Minimum Requirement for the Completion of the Bachelor of Computer Science Honours Degree ?"
query = "who are the lecturers in computer science department with their designation and their qualifications"
docs = db.similarity_search(query)

In [21]:
len(docs)

4

In [22]:
print(docs[0])

page_content='Lecturer  
 
 
 
 
 
 
 
Probationary 
Lecturer  Dr. W. A. Mohotti  
B.Sc.  IT (Moratuwa, SL) 
M.Sc.  in IT (Moratuwa,  SL) 
Ph.D.  (QUT,  Australia)  Data  Mining  and Machine  Learning,  
Text Clustering, Outlier Detection, and 
Cluster Evolution, Social Media Analytics  
Dr. D. W. C. P. Kumari  
B. Sc.  in Comp.  Sci. (UCSC, SL) 
M.Phil.  (UCSC, SL)  
PhD (QUT, Australia) 
(On Leave)  
Mr. K. D. C. G. Kapugama 
BCS (Ruhuna, SL)  
Reading  for PhD  (Monash,  Australia)  
(On Leave)  Information  Security,  Process  
Analytics, Mathematical 
Modelling,  Event  log analysis 
Data Structure & Algorithms 
Evolutionary Algorithms  
Data Mining, Text Mining  
Mr. P. D. T. Chathuranga  
BCS (Ruhuna,  SL) Natural  Language  Processing,  
Sentiment  Analysis,  Text  Mining,  
Machine  Learning  
Mr. L.L. Gihan  Chathuranga  
B.Sc  (Sabaragamuwa,  S.L) Machine  Learning,  Artificial  Neural  
Networks,  Artificial  Intelligence,  
Data  mining  
Ms. H.D.  Supuni  Shashikala  
B.S

In [23]:
docs = db.similarity_search_with_score(query)

In [24]:
docs[0]

(Document(page_content='Lecturer  \n \n \n \n \n \n \n \nProbationary \nLecturer  Dr. W. A. Mohotti  \nB.Sc.  IT (Moratuwa, SL) \nM.Sc.  in IT (Moratuwa,  SL) \nPh.D.  (QUT,  Australia)  Data  Mining  and Machine  Learning,  \nText Clustering, Outlier Detection, and \nCluster Evolution, Social Media Analytics  \nDr. D. W. C. P. Kumari  \nB. Sc.  in Comp.  Sci. (UCSC, SL) \nM.Phil.  (UCSC, SL)  \nPhD (QUT, Australia) \n(On Leave)  \nMr. K. D. C. G. Kapugama \nBCS (Ruhuna, SL)  \nReading  for PhD  (Monash,  Australia)  \n(On Leave)  Information  Security,  Process  \nAnalytics, Mathematical \nModelling,  Event  log analysis \nData Structure & Algorithms \nEvolutionary Algorithms  \nData Mining, Text Mining  \nMr. P. D. T. Chathuranga  \nBCS (Ruhuna,  SL) Natural  Language  Processing,  \nSentiment  Analysis,  Text  Mining,  \nMachine  Learning  \nMr. L.L. Gihan  Chathuranga  \nB.Sc  (Sabaragamuwa,  S.L) Machine  Learning,  Artificial  Neural  \nNetworks,  Artificial  Intelligence,  \nDat

In [25]:
docs[1]

(Document(page_content='B.Sc.  (UWU,  S.L.),  \nReading  for M.Sc.  (Peradeniya,  S.L.)  Machine  Learning,  Feature  Extraction,  \nDeep  Learning  \n1.4 Members  of the Academic  Support  Staff  \n \nAcademic  Supportive  Staff  Members  \nDesignation  Name  \nProgrammer  Cum  Systems  Analyst  Mr. H. G. U. Harankahadeniya  \nMr. B. H. Saranapala  \nMr. A. P. Luwishewa  \nInstructor  in Computer  Technology  Ms. P. B. N. K. De Silva  \nMr. U. V. Malawara  Arachchi \nMs. W.P. Priyanthi  \nMr. R. Wickramaratne \nMr. C.L. Wimalaratne \nMs. G.K. Mabula  \nMr. L.W.  Wellakkage \nMr. G.M.T. Ranjana \nMrs.  W. K. Shajith  \nMiss.  H. G. S. Priyangani  \n \n \n1.5 Course  Units  in Computer  Science  for B.Sc.  (General)  Degree  \nLevel  I - Semester  I \nCOM1111:   Basic  Concepts  of Information  Technology  (15 lecture  hrs.)    Overview  \nof Computer System, Function of Computer System, Input/Output Peripherals, Computer \nStorages, Systems Software, Data type and Data representation, 

In [26]:
docs[2]

(Document(page_content='Rule Modeling  \n• Network  Monitoring  and Acquiring  and Managing  Information  \n• E-commerce information systems development, Model -drivers design, Goal, Business  \n& service Modelling  \n• Computational Geometry, Computer Graphics programming, Design and Analysis of \nAlgorithms, Graph Theory  \n• Computational Systems Biology, Bioinformatics, Modelling and Simulation, Neural \nComputing, stochastic modelling  \n• Embedded  Systems,  reconfigurable  computing,  Bioinformatics  \n• Knowledge Representation, Ontology, Semantic Web, Ontology Engineering, Mobile \nApplications  4  1.2 Head  of the Department  \nDr. W.A.  Indika  \nB.Sc.  (Kelaniya,  S.L.),  M.Sc.  (Kelaniya,  S.L.),  PhD  (UCSC,  S.L.)  \n \n1.3 Members  of Academic  Staff  \n \n \nDesignation  Name  Specialization  \nSenior  \nLecturer  Mr. S. A. S. Lorensuhewa  \nB.Sc.  (Colombo, SL) \nM.Sc.  (Zhejiang,  China)  Computer  Applications,  Text  Mining  and \nText Classification Data Mining, \

In [27]:
docs[3]

(Document(page_content='get experience  on issues  related  to industry.  At the end of the training,  the students  should \nmake a presentation of their findings and submit a report.  \nEvaluation  Method:  Dairy,  Report,  Presentation  \n \n1.8 Bachelor  of Computer  Science  (Honours)  Degree  \nA limited number of students are selected to follow the Bachelor of Computer Science (Hon - \nours) degree programme after completion of three academic years of Bachelor of computer \nscience (General Degree Programme).  The selection of  students is done according to their \nacademic performance during Level I, Level II and Level III of the BCS (General) degree \nprogramme.  \n \n1.8.1  Criteria  for Selection  to the BCS  (Honours)  Degree  Programme  \n1. The Department  of Computer  Science  shall  determine  the number  of students  to be \nadmitted each year.  \n2. A student shall be eligible to apply if he/she has fulfilled the following requirements at \nthe completion of the Level

In [28]:
#docs[4]

### Similarity Check by Vector

In [29]:
embedding_vector= embedding.embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

Lecturer  
 
 
 
 
 
 
 
Probationary 
Lecturer  Dr. W. A. Mohotti  
B.Sc.  IT (Moratuwa, SL) 
M.Sc.  in IT (Moratuwa,  SL) 
Ph.D.  (QUT,  Australia)  Data  Mining  and Machine  Learning,  
Text Clustering, Outlier Detection, and 
Cluster Evolution, Social Media Analytics  
Dr. D. W. C. P. Kumari  
B. Sc.  in Comp.  Sci. (UCSC, SL) 
M.Phil.  (UCSC, SL)  
PhD (QUT, Australia) 
(On Leave)  
Mr. K. D. C. G. Kapugama 
BCS (Ruhuna, SL)  
Reading  for PhD  (Monash,  Australia)  
(On Leave)  Information  Security,  Process  
Analytics, Mathematical 
Modelling,  Event  log analysis 
Data Structure & Algorithms 
Evolutionary Algorithms  
Data Mining, Text Mining  
Mr. P. D. T. Chathuranga  
BCS (Ruhuna,  SL) Natural  Language  Processing,  
Sentiment  Analysis,  Text  Mining,  
Machine  Learning  
Mr. L.L. Gihan  Chathuranga  
B.Sc  (Sabaragamuwa,  S.L) Machine  Learning,  Artificial  Neural  
Networks,  Artificial  Intelligence,  
Data  mining  
Ms. H.D.  Supuni  Shashikala  
B.Sc.  (UWU,  S.L

In [30]:
print(docs[2])

page_content='Rule Modeling  
• Network  Monitoring  and Acquiring  and Managing  Information  
• E-commerce information systems development, Model -drivers design, Goal, Business  
& service Modelling  
• Computational Geometry, Computer Graphics programming, Design and Analysis of 
Algorithms, Graph Theory  
• Computational Systems Biology, Bioinformatics, Modelling and Simulation, Neural 
Computing, stochastic modelling  
• Embedded  Systems,  reconfigurable  computing,  Bioinformatics  
• Knowledge Representation, Ontology, Semantic Web, Ontology Engineering, Mobile 
Applications  4  1.2 Head  of the Department  
Dr. W.A.  Indika  
B.Sc.  (Kelaniya,  S.L.),  M.Sc.  (Kelaniya,  S.L.),  PhD  (UCSC,  S.L.)  
 
1.3 Members  of Academic  Staff  
 
 
Designation  Name  Specialization  
Senior  
Lecturer  Mr. S. A. S. Lorensuhewa  
B.Sc.  (Colombo, SL) 
M.Sc.  (Zhejiang,  China)  Computer  Applications,  Text  Mining  and 
Text Classification Data Mining, 
Rule Extraction and  
Knowledge 

In [31]:
print(docs[3])

page_content='get experience  on issues  related  to industry.  At the end of the training,  the students  should 
make a presentation of their findings and submit a report.  
Evaluation  Method:  Dairy,  Report,  Presentation  
 
1.8 Bachelor  of Computer  Science  (Honours)  Degree  
A limited number of students are selected to follow the Bachelor of Computer Science (Hon - 
ours) degree programme after completion of three academic years of Bachelor of computer 
science (General Degree Programme).  The selection of  students is done according to their 
academic performance during Level I, Level II and Level III of the BCS (General) degree 
programme.  
 
1.8.1  Criteria  for Selection  to the BCS  (Honours)  Degree  Programme  
1. The Department  of Computer  Science  shall  determine  the number  of students  to be 
admitted each year.  
2. A student shall be eligible to apply if he/she has fulfilled the following requirements at 
the completion of the Level III of the Bachelor of C

In [32]:
#print(docs[4])

In [33]:
most_similar_doc = docs[0]

### Generate the Answer

In [34]:
from langchain.llms import OpenAI
llm = OpenAI(api_key = OPENAI_API_KEY)

c:\Users\acer\OneDrive\Desktop\QA\AI_QA\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [35]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm = llm , chain_type = 'stuff')

In [36]:
response = chain.run(input_documents = docs , question = query)

c:\Users\acer\OneDrive\Desktop\QA\AI_QA\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [37]:
response

' \n\n1. Dr. W. A. Mohotti - Probationary Lecturer, B.Sc. IT (Moratuwa, SL), M.Sc. in IT (Moratuwa, SL), Ph.D. (QUT, Australia) in Data Mining and Machine Learning, Text Clustering, Outlier Detection, and Cluster Evolution, Social Media Analytics\n2. Dr. D. W. C. P. Kumari - (On Leave), B.Sc. in Comp. Sci. (UCSC, SL), M.Phil. (UCSC, SL), Ph.D. (QUT, Australia)\n3. Mr. K. D. C. G. Kapugama - (On Leave), BCS (Ruhuna, SL), Reading for PhD (Monash, Australia) in Information Security, Process Analytics, Mathematical Modelling, Event log analysis, Data Structure & Algorithms, Evolutionary Algorithms, Data Mining, Text Mining\n4. Mr. P. D. T. Chathuranga - BCS (Ruhuna, SL) in Natural Language Processing, Sentiment Analysis, Text Mining, Machine Learning\n5. Mr. L.L. Gihan Chathuranga - B.Sc. (Sabaragamuwa'

In [38]:
print(response)

 

1. Dr. W. A. Mohotti - Probationary Lecturer, B.Sc. IT (Moratuwa, SL), M.Sc. in IT (Moratuwa, SL), Ph.D. (QUT, Australia) in Data Mining and Machine Learning, Text Clustering, Outlier Detection, and Cluster Evolution, Social Media Analytics
2. Dr. D. W. C. P. Kumari - (On Leave), B.Sc. in Comp. Sci. (UCSC, SL), M.Phil. (UCSC, SL), Ph.D. (QUT, Australia)
3. Mr. K. D. C. G. Kapugama - (On Leave), BCS (Ruhuna, SL), Reading for PhD (Monash, Australia) in Information Security, Process Analytics, Mathematical Modelling, Event log analysis, Data Structure & Algorithms, Evolutionary Algorithms, Data Mining, Text Mining
4. Mr. P. D. T. Chathuranga - BCS (Ruhuna, SL) in Natural Language Processing, Sentiment Analysis, Text Mining, Machine Learning
5. Mr. L.L. Gihan Chathuranga - B.Sc. (Sabaragamuwa
